In [1]:
from  sklearn.datasets import make_regression
from matplotlib import pyplot
import pandas as pd    
import numpy as np
from DGen import DGenerator

In [2]:
df_cadre = DGenerator.generate_cadres(10, 10, 5, 5, 101)
df_cadre

,cad0,cad1,cad2,cad3,cad4,cad5,cad6,cad7,cad8,cad9,cadre,index_c
0,0.407586,0.807843,0.554055,0.665715,-0.012889,0.417509,0.204408,2.679182,1.203500,0.672022,0,0
1,0.618658,-2.052563,0.452358,0.230087,2.379257,-2.735926,0.456032,0.895160,2.123482,0.763162,0,1
2,-1.087436,-2.567767,1.152164,-1.013228,1.564215,-0.491927,-0.435325,0.893025,2.201165,0.016883,1,0
3,-0.985153,-0.155276,1.368957,-2.736464,1.163777,-1.515968,1.719803,1.135898,0.901734,-0.074192,1,1
4,-0.445873,0.685874,-0.194142,-0.166646,-0.173325,0.030802,-1.303100,-1.512265,-0.138765,0.171330,1,0
5,-0.332921,-0.928602,1.234598,0.661029,-2.033043,-1.818552,0.774966,1.575660,-0.915780,2.713813,2,1
6,1.523750,1.290230,1.147455,2.785010,-2.763423,2.355840,3.443494,-1.776021,-4.382270,-2.827271,2,0
7,-0.140432,0.119145,-3.295186,0.834355,0.319064,1.146729,-1.303806,0.766569,0.205183,0.136841,3,1
8,0.214234,-0.240510,-0.673845,-1.478319,-1.117292,-2.194064,0.803340,-3.791055,-3.135028,0.302713,3,0
9,-0.468756,0.860097,1.465471,1.715950,1.554221,-1.410200,0.565558,-2.076881,0.487243,-1.086860,3,1


In [10]:
z = pd.DataFrame(df_cadre['cadre'])
z

,cadre
0,0
1,0
2,1
3,1
4,1
5,2
6,2
7,3
8,3
9,3


In [3]:
df_variables = DGenerator.generate_variables(df_cadre, 10, 5, 2, 3, 10)
df_variables

,dv0,dv1,dv2,dv3,dv4,target,cadre,index_c
0,2.334594,-1.461914,-1.698423,2.336225,2.416491,0,0,0
1,-2.114044,2.039460,1.854980,-1.164123,1.842182,1,0,1
2,-0.857185,0.288061,0.706234,-0.735791,-1.026204,0,0,2
3,0.608573,-0.323932,-0.905902,-0.461813,-2.244908,0,0,3
4,-0.543823,0.762009,0.269228,-0.718956,-0.238280,1,0,4
5,1.528598,-3.161474,-0.331722,2.723551,0.664783,0,0,5
6,-0.911076,-2.195991,1.863004,1.105104,-0.649393,1,0,6
7,0.110839,0.255052,-0.565296,-0.932984,-2.170261,0,0,7
8,-2.870849,2.951982,2.200476,-2.275953,0.915428,1,0,8
9,-0.136675,0.612532,-0.377733,-1.104456,-1.824018,1,0,9


In [7]:
b = len(df_cadre['cadre'].unique())
b

4

In [4]:
f = df_cadre['cadre']
f.astype(int)
f

0    0
6    0
1    1
2    1
7    1
3    2
9    2
4    3
5    3
8    3
Name: cadre, dtype: int32